# Using Paddle Quantum

In this notebook, we give an example of using the Paddle Quantum framework for simulating MBQC patterns. 


An introduction to Paddle Quantum's MBQC simulator, along with the relevant documentation, can be found [here](https://qml.baidu.com/tutorials/measurement-based-quantum-computation/mbqc-quick-start-guide.html). 

In [1]:
from paddle_quantum.mbqc.simulator import MBQC
from paddle_quantum.mbqc.qobject import State
from paddle_quantum.mbqc.utils import basis
from paddle import to_tensor
import numpy as np
import random as rand

C:\Users\joeh1\anaconda3\lib\site-packages\paddle\tensor\creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


For our purposes we will consider the implementation of the 10-qubit 'brick' state: 

<img src="brick.png" width=350px>

where numbers simply label the qubits. For our simulations we encode this graph in the following way:

In [2]:
# List of vertices as strings
V = [str(v) for v in range(10)]

# List of edges as pairs of strings
E = [('0','2'),('1','3'),('2','4'),('3','5'),('4','5'),('4','6'),('5','7'),
    ('6','8'),('7','9'),('8','9')]

# Define graph
G = [V,E]

We also need to define a list of measurement angles. In this case we opt to implement the 10-qubit controlled-$X$ gate: 
<img src="brick-cnot.png" width=350px>
Each angle $\theta$ is with respect to the basis $M_{XY}(\theta) = \{ |+_\theta\rangle, |-_\theta\rangle \}$ where $|\pm_\theta\rangle = \frac{1}{\sqrt{2}}\left( |0\rangle \pm e^{i\theta} |1\rangle\right)$. Hence we define the list of angles as follows:

In [3]:
pi = np.pi
mmt_angles = [0,0,0,pi/2,pi/2,0,0,-pi/2,0,0]

To show this works, we first simulate the pattern without applying the UBQC protocol. We start by defining the input state of the pattern to be $|11\rangle$, so that the output state should be $|10\rangle$. The formatting is again a bit awkward:

In [4]:
# i_vs = [rand.choice([0,1]) for i in range(2)]
# client_input_state = [[],[]]
# for i in range(2):
#     if i_vs[i] == 0:
#         client_input_state[i] = [0,1]
#     else:
#         client_input_state[i] = [1,0]
# input_state = np.kron(client_input_state[0], client_input_state[1]) 
input_state = [0,0,0,1]
input_state = to_tensor([[x] for x in input_state], dtype='complex128')
input_state = State(input_state, ['0','1'])

We then initialise the MBQC class, setting the graph state and input state. 

In [5]:
mbqc = MBQC()
mbqc.set_graph(G)
mbqc.set_input_state(input_state)

All that's left is the measurements and angle corrections. We first define the sets $s_X$ and $s_Z$ for each vertex, as standard in MBQC. 

In [6]:
sx_sets = [[],[],[0],[1],[2],[3],[4],[5],[6],[7]]
sz_sets = [[],[],[],[],[0,3],[1,2],[2],[3],[4,7],[5,6]]

We then measure each non-output vertex, first computing the measurement corrections where necessary. 

In [7]:
for i in range(8):
    # Input vertices do not have angle corrections
    if i < 2:
#         theta = ((-1)**(i_vs[i]))*mmt_angles[i]
        theta = mmt_angles[i]
    else:
        sx = [str(x) for x in sx_sets[i]]
        sz = [str(x) for x in sz_sets[i]]
        sx_sum = mbqc.sum_outcomes(sx)
        sz_sum = mbqc.sum_outcomes(sz)
        theta = ((-1)**sx_sum) * mmt_angles[i] + sz_sum*pi
        
        
    # Reformat measurement angle
    theta = to_tensor(theta, dtype='float64')
    mbqc.measure(str(i), basis('XY', theta))

Finally we perform the appropriate gate correction to the output qubits before measuring them in the $Z$-basis. 

In [8]:
# Gate corrections
for i in [8,9]:
    sx = [str(x) for x in sx_sets[i]]
    sz = [str(x) for x in sz_sets[i]]
    sx_sum = mbqc.sum_outcomes(sx)
    sz_sum = mbqc.sum_outcomes(sz)
    mbqc.correct_byproduct('Z', str(i), sz_sum)
    mbqc.correct_byproduct('X', str(i), sx_sum)

# Measure output qubits in Z basis
for i in [8,9]:
    zero = to_tensor(0, dtype='float64')
    mbqc.measure(str(i), basis('YZ', zero))
    
# Print output measurement outcomes
mbqc.sum_outcomes(['8']), mbqc.sum_outcomes(['9'])

(1, 0)

... and we get the right result!